## Interactive plotting for large datasets using Gaia

#### This (currently unfinished) demo will show you how to use vaex or holoviews to render very large datasets interactively.

In [ ]:
gaia_file = 'rv-sort-by-source_id.hdf5' 

### vaex:

In [ ]:
import vaex
ds = vaex.open(gaia_file)

In [ ]:
len(ds)

In [ ]:
ds.plot("ra","dec", f="log1p")

In [ ]:
import vaex.jupyter
ds.select(ds['radial_velocity' != 0])
@vaex.jupyter.interactive_selection(ds)
def plot():
    print("Mean RV for the selection is:", ds.mean(ds['radial_velocity'], selection=True))
    ds.plot("ra", "dec", what=np.log(vaex.stat.count()+1), selection=[None, True])
    plt.show()

###### Here are some resources that may be of use:
- [Vaex tutorial](https://github.com/maartenbreddels/vaex/blob/master/docs/source/tutorial.ipynb)
- [Vaex demo notebooks](https://github.com/maartenbreddels/vaex/tree/master/examples)
- [a little bit of Vaex stuff from Lauren Anderson](https://github.com/andersdot/gaiaDR2/blob/master/playVaex.ipynb)

### holoviews/datashader:

In [ ]:
import holoviews as hv
hv.notebook_extension('bokeh')

In [ ]:
import pandas as pd
import h5py
from astropy.table import Table

data = Table()
with h5py.File(gaia_file) as f:
    #columns = list(f['table/columns'])
    columns = ['ra', 'dec', 'pmra', 'pmdec', 'parallax', 'radial_velocity', 'radial_velocity_error']
    for c in columns: data[c] = f['table/columns/{0}/data'.format(c)][:]

In [ ]:
ds = hv.Dataset(data.to_pandas())

In [ ]:
len(ds)

In [ ]:
%%opts RGB [width=800, height=500]
from holoviews.operation.datashader import datashade, dynspread
import datashader

datashade(hv.Points(ds, kdims=['ra', 'dec'], vdims=['parallax']), aggregator=datashader.mean('parallax'))

###### Here are some resources that may be of use:
- [OpenSky example with datashader](https://anaconda.org/jbednar/opensky/notebook)
- get holoviews example ipython notebooks:
```
holoviews --install-examples
cd holoviews-examples
```
- [Gaia example from Tim Morton](https://github.com/timothydmorton/gaia-viz-demo/blob/master/demo.ipynb)